In [70]:
import pandas as pd
import numpy as np
import math
import swifter #used to make pd.apply more efficient
import langid
from langid.langid import LanguageIdentifier, model # used to analyze the language that the song is in
#from langdetect import detect
import matplotlib.pyplot as plt
%matplotlib inline  
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.classify import textcat
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer
from wordcloud import WordCloud, STOPWORDS
from textblob import TextBlob

In [2]:
tracks_df = pd.read_csv('./data/all_tracks_cleaned.csv')

In [3]:
tracks_df.head()

,track_uri,track_name,cleaned_track_name,track_artists,featured_artists,track_is_explicit,track_popularity,track_genres,track_duration_ms,track_time_signature,...,track_valence,track_lyrics,lyrics_page_views,track_number,album_name,album_artist,album_release_date,album_popularity,album_record_label,album_cover
0,spotify:track:2HSmyk2qMN8WQjuGhaQgCk,Champagne Poetry,Champagne Poetry,"('Drake',)",NaN,True,82,"('hip hop', 'pop', 'rap')",336511,4.0,...,0.496,"Champagne Poetry Lyrics\n\nI love you, I love ...",688853.0,1,Certified Lover Boy,Drake,2021-09-03,95,OVO,https://i.scdn.co/image/ab67616d00001e02cd945b...
1,spotify:track:6jy9yJfgCsMHdu2Oz4BGKX,Papi’s Home,Papi’s Home,"('Drake',)",NaN,True,76,"('hip hop', 'pop', 'rap')",178623,4.0,...,0.588,Papi’s Home Lyrics\nI know that I hurt you\nYe...,445883.0,2,Certified Lover Boy,Drake,2021-09-03,95,OVO,https://i.scdn.co/image/ab67616d00001e02cd945b...
2,spotify:track:37Nqx7iavZpotJSDXZWbJ3,Girls Want Girls (with Lil Baby),NaN,"('Drake', 'Lil Baby')",NaN,True,86,"('hip hop', 'pop', 'rap', 'trap')",221979,4.0,...,0.381,NaN,NaN,3,Certified Lover Boy,Drake,2021-09-03,95,OVO,https://i.scdn.co/image/ab67616d00001e02cd945b...
3,spotify:track:61S79KIVA4I9FXbnsylEHT,In The Bible (with Lil Durk & Giveon),In The Bible,"('Drake', 'Lil Durk', 'Giveon')","('GIVĒON', 'Lil Durk')",True,79,"('hip hop', 'pop', 'rap', 'drill', 'trap', 'r&b')",296568,4.0,...,0.147,"In The Bible Lyrics\nOkay, okay, okay\nCountin...",439186.0,4,Certified Lover Boy,Drake,2021-09-03,95,OVO,https://i.scdn.co/image/ab67616d00001e02cd945b...
4,spotify:track:4VCbgIdr8ptegWeJpqLVHH,Love All (with JAY-Z),NaN,"('Drake', 'JAY-Z')",NaN,True,77,"('hip hop', 'pop', 'rap')",228461,4.0,...,0.155,NaN,NaN,5,Certified Lover Boy,Drake,2021-09-03,95,OVO,https://i.scdn.co/image/ab67616d00001e02cd945b...


In [4]:
tracks_df['track_lyrics'][1000] # before cleaning

"Yummy Lyrics\nYeah, you got that yummy-yum\nThat yummy-yum, that yummy-yummy\nYeah, you got that yummy-yum\nThat yummy-yum, that yummy-yummy\nSay\u2005the\u2005word, on my\u2005way\nYeah, babe, yeah, babe, yeah, babe\nAny\u2005night, any day\nSay the word, on my way\nYeah, babe, yeah, babe, yeah, babe\nIn the mornin' or the late\nSay the word, on my way\n\nBona fide stallion\nAin't in no stable, no, you stay on the run\nAin't on the side, you're number one\nYeah, every time I come around, you get it done\n\nFifty-fifty, love the way you split it\nHundred racks, help me spend it, babe\nLight a match, get litty, babe\nThat jet set, watch the sunset kinda, yeah, yeah\nRollin' eyes back in my head, make my toes curl, yeah, yeah\nYeah, you got that yummy-yum\nThat yummy-yum, that yummy-yummy\nYeah, you got that yummy-yum\nThat yummy-yum, that yummy-yummy\nSay the word, on my way\nYeah, babe, yeah, babe, yeah, babe\nAny night, any day\nSay the word, on my way\nYeah, babe, yeah, babe, yeah, 

In [5]:
tracks_df['track_lyrics'] = tracks_df['track_lyrics'].str.replace(u"\u2005"," ").str.replace(u"\u205f"," ") #replace any \u characters 
tracks_df['track_lyrics'] = tracks_df['track_lyrics'].str.replace('\n', ' ').str.replace("\'", "") # removing \n and \
tracks_df['track_lyrics'] = tracks_df['track_lyrics'].str.replace(r'[\[].*?[\]]','', regex=True) # removing section dividers [chorus] [intro] if any exist

In [6]:
tracks_df['track_lyrics'][1000] # after cleaning

'Yummy Lyrics Yeah, you got that yummy-yum That yummy-yum, that yummy-yummy Yeah, you got that yummy-yum That yummy-yum, that yummy-yummy Say the word, on my way Yeah, babe, yeah, babe, yeah, babe Any night, any day Say the word, on my way Yeah, babe, yeah, babe, yeah, babe In the mornin or the late Say the word, on my way  Bona fide stallion Aint in no stable, no, you stay on the run Aint on the side, youre number one Yeah, every time I come around, you get it done  Fifty-fifty, love the way you split it Hundred racks, help me spend it, babe Light a match, get litty, babe That jet set, watch the sunset kinda, yeah, yeah Rollin eyes back in my head, make my toes curl, yeah, yeah Yeah, you got that yummy-yum That yummy-yum, that yummy-yummy Yeah, you got that yummy-yum That yummy-yum, that yummy-yummy Say the word, on my way Yeah, babe, yeah, babe, yeah, babe Any night, any day Say the word, on my way Yeah, babe, yeah, babe, yeah, babe In the mornin or the late Say the word, on my way  

In order to identify the language that the song is in, a function is created which takes each lyric and classifies its lyrics using lang_identify.classify(). A list of all the unique languages is made in order to make the process of cleaning easier.

In [50]:
languages = []
lang_identify = LanguageIdentifier.from_modelstring(model, norm_probs=True)
def detect_language(lyrics):
    #print(lyrics)
    if np.any(pd.notna(lyrics)):
        lang = lang_identify.classify(lyrics)
    else:
        lang = np.nan
        
    if lang == np.nan:
        return lang
    else: 
        if type(lang) == float:
            return 
        else:
            if lang[0] not in languages:
                languages.append(lang[0])
            return lang[0]
        

Apply function on the df and use swifter library which uses builtin multithreading to speadup the process. The list of unique languages is printed.

In [51]:
lang_detected = tracks_df.swifter.apply(lambda x: detect_language(x.track_lyrics), axis=1)
#tc = textcat.TextCat()
#print(tracks_df['track_lyrics'][0:10].apply(tc.guess_language))
#tracks_df[0:10].apply(lambda x: tc.guess_language(x.track_lyrics) if(np.all(pd.notna(x))) else x, axis=1).head
print(languages)

Pandas Apply: 100%|██████████| 156480/156480 [08:22<00:00, 311.24it/s] 

['en', 'ja', 'it', 'es', 'ko', 'pt', 'de', 'fr', 'br', 'zh', 'hi', 'no', 'la', 'ms', 'fa', 'sw', 'id', 'cy', 'fi', 'hu', 'jv', 'lb', 'lv', 'af', 'an', 'is', 'zu', 'nl', 'mg', 'cs', 'ur', 'et', 'da', 'oc', 'sv', 'se', 'tl', 'ru', 'gl', 'ga', 'nn', 'eo', 'mt', 'el', 'he', 'rw', 'ky', 'vo', 'tr', 'pl', 'qu', 'lt', 'ps']


Insert language column into dataframe

In [52]:
tracks_df.insert(4, 'lyrics_language', lang_detected)
tracks_df

,track_uri,track_name,cleaned_track_name,track_artists,lyrics_language,featured_artists,track_is_explicit,track_popularity,track_genres,track_duration_ms,...,track_valence,track_lyrics,lyrics_page_views,track_number,album_name,album_artist,album_release_date,album_popularity,album_record_label,album_cover
0,spotify:track:2HSmyk2qMN8WQjuGhaQgCk,Champagne Poetry,Champagne Poetry,"('Drake',)",en,NaN,True,82,"('hip hop', 'pop', 'rap')",336511,...,0.496,"Champagne Poetry Lyrics I love you, I love yo...",688853.0,1,Certified Lover Boy,Drake,2021-09-03,95,OVO,https://i.scdn.co/image/ab67616d00001e02cd945b...
1,spotify:track:6jy9yJfgCsMHdu2Oz4BGKX,Papi’s Home,Papi’s Home,"('Drake',)",en,NaN,True,76,"('hip hop', 'pop', 'rap')",178623,...,0.588,Papi’s Home Lyrics I know that I hurt you Yeah...,445883.0,2,Certified Lover Boy,Drake,2021-09-03,95,OVO,https://i.scdn.co/image/ab67616d00001e02cd945b...
2,spotify:track:37Nqx7iavZpotJSDXZWbJ3,Girls Want Girls (with Lil Baby),NaN,"('Drake', 'Lil Baby')",None,NaN,True,86,"('hip hop', 'pop', 'rap', 'trap')",221979,...,0.381,NaN,NaN,3,Certified Lover Boy,Drake,2021-09-03,95,OVO,https://i.scdn.co/image/ab67616d00001e02cd945b...
3,spotify:track:61S79KIVA4I9FXbnsylEHT,In The Bible (with Lil Durk & Giveon),In The Bible,"('Drake', 'Lil Durk', 'Giveon')",en,"('GIVĒON', 'Lil Durk')",True,79,"('hip hop', 'pop', 'rap', 'drill', 'trap', 'r&b')",296568,...,0.147,"In The Bible Lyrics Okay, okay, okay Countin u...",439186.0,4,Certified Lover Boy,Drake,2021-09-03,95,OVO,https://i.scdn.co/image/ab67616d00001e02cd945b...
4,spotify:track:4VCbgIdr8ptegWeJpqLVHH,Love All (with JAY-Z),NaN,"('Drake', 'JAY-Z')",None,NaN,True,77,"('hip hop', 'pop', 'rap')",228461,...,0.155,NaN,NaN,5,Certified Lover Boy,Drake,2021-09-03,95,OVO,https://i.scdn.co/image/ab67616d00001e02cd945b...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156475,spotify:track:0VxTtE5HoNMf9sp30j6c9V,Try Again,NaN,"('Westlife',)",None,NaN,False,47,"('boy band', 'dance pop', 'europop')",214866,...,0.381,NaN,NaN,14,Westlife,Westlife,1999-11-01,73,RCA Records Label,https://i.scdn.co/image/ab67616d00001e0244ead2...
156476,spotify:track:3EHx4H0FsTplZrcFSeuLeE,What I Want Is What I Got,What I Want Is What I’ve Got,"('Westlife',)",en,NaN,False,46,"('boy band', 'dance pop', 'europop')",213066,...,0.744,What I Want Is What I’ve Got Lyrics All that I...,NaN,15,Westlife,Westlife,1999-11-01,73,RCA Records Label,https://i.scdn.co/image/ab67616d00001e0244ead2...
156477,spotify:track:4GfGx2zvY8pIwf2o2SAufU,We Are One,NaN,"('Westlife',)",None,NaN,False,45,"('boy band', 'dance pop', 'europop')",222893,...,0.426,NaN,NaN,16,Westlife,Westlife,1999-11-01,73,RCA Records Label,https://i.scdn.co/image/ab67616d00001e0244ead2...
156478,spotify:track:7dODnrD8danC9FD5xLb9Tu,Can't Lose What You Never Had,Can’t Lose What You Never Had,"('Westlife',)",en,NaN,False,45,"('boy band', 'dance pop', 'europop')",264485,...,0.656,Can’t Lose What You Never Had Lyrics Baby your...,6644.0,17,Westlife,Westlife,1999-11-01,73,RCA Records Label,https://i.scdn.co/image/ab67616d00001e0244ead2...


Get the counts of each language.

In [53]:
tracks_df.lyrics_language.value_counts()

en    70655
es     8681
de     1149
fr     1050
pt      967
ko      604
it      142
ja      129
la       90
id       55
sw       44
zh       26
sv       24
gl       24
br       22
qu       20
ga       20
an       19
tl       18
af       12
lb       11
nl       10
ms       10
jv       10
da        9
tr        8
cy        8
hi        7
mg        7
mt        5
oc        5
ky        5
eo        5
lv        4
lt        4
fi        4
ps        3
ru        3
hu        2
et        2
he        2
se        2
vo        2
zu        2
pl        2
no        2
ur        2
nn        1
is        1
el        1
rw        1
fa        1
cs        1
Name: lyrics_language, dtype: int64

Now it is important to check which languages are supported by the nltk library to clean the lyrics.

In [71]:
from nltk.corpus import stopwords
print(stopwords.fileids())

['arabic', 'azerbaijani', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']


 Below is a dictionary of the languages abbreviations and their actual names. This dictionary only contains the  languages supported by the nltk library.

In [54]:
language_short_name = {'ar':'arabic', 'az':'azerbaijani', 'da':'danish','de':'german', 'nl':'Dutch; Flemish', 'el':'Greek-Modern (1453-)',
'en':'english', 'fi':'finnish','fr':'French', 'el':'Greek-Modern (1453-)', 'hu':'hungarian', 'id':'indonesian', 'it':'italian', 'ja':'japanese',
'kk':'kazakh', 'ne':'Nepali','nl':'Dutch; Flemish','nn':'Norwegian Nynorsk; Nynorsk:Norwegian', 'nb':'Bokmål-Norwegian; Norwegian Bokmål',
'no':'Norwegian','pt':'portuguese','ro':'Romanian; Moldavian; Moldovan','ru':'Russian','sl':'Slovenian','es':'Spanish; Castilian','sv':'Swedish',
'tg':'Tajik','tr':'Turkish'}

In [56]:
for lang in languages:
    print(language_short_name.get(lang))

English
Japanese
Italian
Spanish; Castilian
Korean
Portuguese
German
French
Breton
Chinese
Hindi
Norwegian
Latin
Malay
Persian
Swahili
Indonesian
Welsh
Finnish
Hungarian
Javanese
Luxembourgish; Letzeburgesch
Latvian
Afrikaans
Aragonese
Icelandic
Zulu
Dutch; Flemish
Malagasy
Czech
Urdu
Estonian
Danish
Occitan (post 1500)
Swedish
Northern Sami
Tagalog
Russian
Galician
Irish
Norwegian Nynorsk; Nynorsk:Norwegian
Esperanto
Maltese
Greek-Modern (1453-)
Hebrew
Kinyarwanda
Kirghiz; Kyrgyz
Volapük
Turkish
Polish
Quechua
Lithuanian
Pushto; Pashto


In [91]:
# TODO identify if the lyrics are catagorized correctly

# remove fa, ur, rw, zw, vo, he, ru, ko, zh, ja (these were checked)

# multi lingual: el, he, ru XXXTENTACION - I don’t even speak spanish lol ..., ru To Have and to Hold Lyrics "В докладах рассмат...
## ps french and persian 
# is, se, lt to es
# nn, no, pl, cs, et, to en
# GET LOOSE NOW Lyrics Five, six, seven, eight ... to en
# I Want You Now Lyrics I want you now, tomorrow... to en
# Hava LyricsAre you ready? Are you ready?Embed to en
# Ay-Oh (Live Aid) LyricsAy-oh (Ay-oh) Ay-oh (Ay... to en
# You Stayed… (RS Home Demo 8/82) Lyrics.Embed to en
# Wild Honey Pie (2018 Mix) Lyrics Honey pie Hon... to en
# Āfer Ventus Lyrics Mare Nūbium, Umbriel (Suus... to la (lv)

x = tracks_df[tracks_df['lyrics_language'].str.match('sw', na=False)]
x[['lyrics_language','track_lyrics']].to_csv('korean.csv')

In [62]:
# de
# "Beauties Can Die Lyrics Embed" 1
# Desiderium 207 LyricsEmbed 4
#Flim Flam LyricsEmbed 1
# Foreign LyricsEmbed 1
# Rich Porter (Skit) LyricsEmbed

# Robot Rock (Maximum Overdrive Mix) LyricsRock Robot Rock Embed english
# Enter Sandman (July 6th, 1990, Writing In Progress) Lyrics : Yeah!  shit Yeah! Hey! Hey! *finger snapping* *drums being played*  Embed
# Seattle Yodel LyricsDiddly-day-hee, odeleo-hee, odeleo-day-day-day-odeleo-hee Diddly-day-hee, odeleo-hee, odeleo-day-day-day-odeleo-hee (Diddly-day-hee, odeleo-hee, odeleo-day-day-day-odeleo-hee) Diddly-day-hee, odeleo-hee, odeleo-day-day-day-odeleo-hee (Diddly-day-hee, odeleo-hee, odeleo-day-day-day-odeleo-hee) (Diddly-day-hee, odeleo-hee, odeleo-day-day-day-odeleo-hee) (Diddly-day-hee, odeleo-hee, odeleo-day-day-day-odeleo-hee)2Embed


# def detect_languages(lyrics):
#     print(lyrics)
#     if lyrics == None:
#         return
#     #tc = textcat.TextCat()
#     #lang = lyrics.apply(tc.guess_language)
#     lang = detect(lyrics)
#     if lang not in languages:
#         languages.append(language_short_name.get(lang))